# dislocation_periodic_array calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

Description updated: 2019-07-26

## Introduction

The dislocation_periodic_array calculation constructs an atomic system with a periodic array of dislocations configuration.  A single dislocation is inserted into an otherwise perfect crystal, and the system is kept periodic in the two system box directions that are within the dislocation's slip plane.  The system is then statically relaxed with the atoms at the boundary perpendicular to the slip plane held fixed.  The generated system can then be used by the other "dislocation_periodic_array_*" calculations for examining the slip response of a dislocation to applied stresses or strain rates.

### Version notes

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- This calculation was designed to be general enough to properly generate a dislocation for any crystal system, but has so far only been tested with cubic systems.  The resulting system should be carefully examined if the base system in which the dislocation is inserted is not orthorhombic.  In particular, the method may need adjusting if the $\vec{c}$ box vector has a large $y$ component.


## Method and Theory

### System orientation considerations

Properly constructing a periodic array of dislocations atomic configuration requires careful consideration of dislocation solutions and atomic system boundaries.  Solutions for straight dislocations based on elasticity often follow the convention of using a Cartesian system ($x', y', z'$) in which the dislocation line is oriented along the $z'$-axis, and the slip plane taken to be the $y'=0$ plane. The dislocation's Burgers vector, $\vec{b}$, is then in the $x'z'$-plane, with edge component in the $x'$-direction and screw component in the $z'$-direction.  When the dislocation slips, the dislocation line will move in the $x'$-direction.

For any such dislocation solution, there will be a shearing along the slip plane resulting in disregistry, i.e. a relative displacement between the top and bottom halves.  This disregistry has limits such that it is $0$ for $x' \to -\infty$ and $\vec{b}$ for $x' \to +\infty$.

Within an atomic system, the dislocation line should be aligned with one of the system's box vectors making the dislocation infinitely long and initially perfectly straight.  The slip plane can then be defined as containing that box vector and another one.  This results in the third box vector being the only one with a component parallel to the slip plane's normal.

For LAMMPS-based simulations, the most convenient orientation to use is to align the dislocation with the $\vec{a}$ box vector, and to define the slip plane as containing both $\vec{a}$ and $\vec{b}$.  Given the limits that LAMMPS places on how system boxes can be defined, this results in favorable alignment of the system to the LAMMPS Cartesian system ($x, y, z$). The dislocation line will be along the $x$-axis, the slip plane normal parallel to the $z$-axis, and dislocation motion will be in the $y$ direction. Thus, the LAMMPS coordinates corresponds to a rotation of the theory coordinates such that $x'=y, y'=z, z'=x$.

### Linear displacements solution

The simplest way to insert a dislocation is to cut the system in half along the slip plane and apply equal but opposite linear displacements, $\vec{u}$, to the two halves with end conditions

- $\vec{u}(y=-\frac{Ly}{2}) = 0$
- $\vec{u}(y=\frac{Ly}{2}) = \pm \frac{\vec{b}}{2}$

Applying these displacements results in a disregistry along the slip plane that ranges from $0$ to $\vec{b}$.  While the two $y$ boundaries of the system both correspond to a perfect crystal, they are misaligned from each other by $\frac{\vec{b}}{2}$.  A coherent periodic boundary along the $\vec{b}$ box vector can be established by adding or subtracting $\frac{\vec{b}}{2}$ from $\vec{b}$.  

Note that with dislocations containing an edge component, a half-plane of atoms either needs to be inserted or removed to ensure boundary compatibility. Here, this is accomplished by always shifting $\vec{b}$ to be shorter in the $y$ direction, and removing excess atoms by identifying (near) duplicates.

### Using dislocation solutions

A slightly more complicated, but ultimately more efficient, way of creating a periodic array of dislocations system is to combine the linear displacements solultion above with a more accurate linear elastic dislocation solution.  The linear solution is used for the atoms at the free surfaces in the $z$ direction, and for ensuring periodicity across the $\vec{b}$ box vector direction.  The linear elastic dislocation solution is then used for atoms in the middle of the system to construct an initial dislocation.


## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import random
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://matplotlib.org/
import matplotlib.pyplot as plt
%matplotlib inline

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2020-04-01 using iprPy version 0.10.0


#### 1.2. Default calculation setup

In [7]:
# Specify calculation style
calc_style = 'dislocation_periodic_array'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)
    
# Initialize connection to library
library = iprPy.Library()

### 2. Assign values for the calculation's run parameters

#### 2.1. Specify system-specific paths

- __lammps_command__ (required) is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [8]:
lammps_command = 'C:/Program Files/LAMMPS/2020-03-03/bin/lmp_serial.exe'
mpi_command = None

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [9]:
potential_name = '2016--Wilson-S-R--Mg--LAMMPS--ipr1'

# Retrieve potential and parameter file(s)
potential = library.get_potential(id=potential_name, get_files=True, verbose=True)

1 matching LAMMPS potentials found from remote database
Files for 0 LAMMPS potentials copied
Files for 1 LAMMPS potentials downloaded


#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load parameters and symbols.

In [10]:
prototype_name = 'A3--Mg--hcp'
symbols = ['Mg']
box_parameters = uc.set_in_units([3.1853, 3.1853, 5.1853], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = library.get_ref(style='crystal_prototype', name=prototype_name)

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], gamma=120, scale=True)

print(ucell)

avect =  [ 3.185,  0.000,  0.000]
bvect =  [-1.593,  2.759,  0.000]
cvect =  [ 0.000,  0.000,  5.185]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 1
symbols = ('Mg',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   1.839 |   2.593


#### 2.4 Specify material elastic constants

Simple input parameters:

- __C_dict__ is a dictionary containing the unique elastic constants for the potential and crystal structure defined above. 

Derived parameters

- __C__ is an atomman.ElasticConstants object built from C_dict.

In [11]:
C_dict = {}
C_dict['C11'] = uc.set_in_units(70.0, 'GPa')
C_dict['C12'] = uc.set_in_units(26.742, 'GPa')
C_dict['C13'] = uc.set_in_units(15.35, 'GPa')
C_dict['C44'] = uc.set_in_units(124.84, 'GPa')
C_dict['C33'] = uc.set_in_units(68.97, 'GPa')


# -------------- Derived parameters -------------- #
# Build ElasticConstants object from C_dict terms
C = am.ElasticConstants(**C_dict)
print(uc.get_in_units(C.Cij, 'GPa'))

[[ 70.     26.742  15.35    0.      0.      0.   ]
 [ 26.742  70.     15.35    0.      0.      0.   ]
 [ 15.35   15.35   68.97    0.      0.      0.   ]
 [  0.      0.      0.    124.84    0.      0.   ]
 [  0.      0.      0.      0.    124.84    0.   ]
 [  0.      0.      0.      0.      0.     21.629]]


#### 2.5 Specify the defect parameters

- __dislocation_name__ gives the name of the dislocation_monopole reference record in the iprPy library to use for potential. 

- __dislocation_file__ gives the path to a dislocation_monopole reference containing defect input parameters. Here, this is built automatically using dislocation_name and librarydir.

- __dislocation_kwargs__ is a dictionary containing parameters for generating the defect. Values are extracted from the dislocation_model record and uniquely define a type of stacking fault. Included keywords are:

    - __burgers__ is the crystallographic Burgers vector for the dislocation
    
    - __a_uvw, b_uvw, c_uvw__ specifies how to orient the system by defining the crystal vectors of ucell to place along the three system box vectors.
    
    - __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.
    
    - __stroh_m__ a Cartesian unit vector used by the Stroh method to define the created dislocation's orientation and type. This vector is in the dislocation's slip plane and perpendicular to the dislocation's line direction. Default value is \[0, 1, 0\], i.e. along the y-axis. 
    
    - __stroh_n__ a Cartesian unit vector used by the Stroh method to define the created dislocation's orientation and type. This vector is normal to the dislocation's slip plane. Default value is \[0, 0, 1\], i.e. along the z-axis.
    
    - __lineboxvector__ the system box vector, 'a', 'b', or 'c', along which the dislocation line is placed parallel to. Default value is 'a'.

In [12]:
dislocation_file = DM()
dislocation_file['dislocation'] = dislparams = DM()

dislparams['key'] = 'unset'
dislparams['id'] = 'A3--Mg--hcp--a-3-1120--90-edge--{0001}'
dislparams['character'] = 'edge'
dislparams['Burgers-vector'] = "a/3[ 1, 1, -2, 0]"
dislparams['slip-plane'] = [ 0, 0, 0, 1]
dislparams['line-direction'] = [ 1, -1, 0, 0]
dislparams['system-family'] = 'A3--Mg--hcp'
dislparams['calculation-parameter'] = cp = DM()

cp['stroh_m'] = '0 1 0'
cp['stroh_n'] = '0 0 1'
cp['lineboxvector'] = 'a'
cp['a_uvw'] = " 1 -1 0 0"
cp['b_uvw'] = "1 1 -2 0 "
cp['c_uvw'] = " 0 0 0 1 "
cp['atomshift'] = '0.00 0.00 0.05'
cp['burgersvector'] = "  0.333 0.333 -0.666 0"

print(dislocation_file.json(indent=4))

{
    "dislocation": {
        "key": "unset", 
        "id": "A3--Mg--hcp--a-3-1120--90-edge--{0001}", 
        "character": "edge", 
        "Burgers-vector": "a/3[ 1, 1, -2, 0]", 
        "slip-plane": [
            0, 
            0, 
            0, 
            1
        ], 
        "line-direction": [
            1, 
            -1, 
            0, 
            0
        ], 
        "system-family": "A3--Mg--hcp", 
        "calculation-parameter": {
            "stroh_m": "0 1 0", 
            "stroh_n": "0 0 1", 
            "lineboxvector": "a", 
            "a_uvw": " 1 -1 0 0", 
            "b_uvw": "1 1 -2 0 ", 
            "c_uvw": " 0 0 0 1 ", 
            "atomshift": "0.00 0.00 0.05", 
            "burgersvector": "  0.333 0.333 -0.666 0"
        }
    }
}


In [13]:
dislocation_file = DM()
dislocation_file['dislocation'] = dislparams = DM()

dislparams['key'] = 'unset'
dislparams['id'] = 'A3--Mg--hcp--a-3-1120--90-edge--{0001}'
dislparams['character'] = 'edge'
dislparams['Burgers-vector'] = "a/3[ 1, 1, -2, 0]"
dislparams['slip-plane'] = [ 0, 0, 0, 1]
dislparams['line-direction'] = [ 1, -1, 0, 0]
dislparams['system-family'] = 'A3--Mg--hcp'
dislparams['calculation-parameter'] = cp = DM()

cp['stroh_m'] = '0 1 0'
cp['stroh_n'] = '0 0 1'
cp['lineboxvector'] = 'a'
cp['a_uvw'] = " 1 -1  0"
cp['b_uvw'] = " 1  1  0 "
cp['c_uvw'] = " 0  0  1"
cp['atomshift'] = '0.00 0.00 0.05'
cp['burgersvector'] = "1 1 0"

print(dislocation_file.json(indent=4))

{
    "dislocation": {
        "key": "unset", 
        "id": "A3--Mg--hcp--a-3-1120--90-edge--{0001}", 
        "character": "edge", 
        "Burgers-vector": "a/3[ 1, 1, -2, 0]", 
        "slip-plane": [
            0, 
            0, 
            0, 
            1
        ], 
        "line-direction": [
            1, 
            -1, 
            0, 
            0
        ], 
        "system-family": "A3--Mg--hcp", 
        "calculation-parameter": {
            "stroh_m": "0 1 0", 
            "stroh_n": "0 0 1", 
            "lineboxvector": "a", 
            "a_uvw": " 1 -1  0", 
            "b_uvw": " 1  1  0 ", 
            "c_uvw": " 0  0  1", 
            "atomshift": "0.00 0.00 0.05", 
            "burgersvector": "1 1 0"
        }
    }
}


In [14]:
am.tools.miller.vector4to3([1, -1, 0, 0])

array([ 1., -1.,  0.])

In [15]:
am.tools.miller.vector4to3([1, 1, -2, 0])

array([3., 3., 0.])

In [16]:
am.tools.miller.vector4to3([0, 0, 0, 1])

array([0., 0., 1.])

In [30]:
print(am.tools.miller.vector4to3([1/3, 1/3, -2/3, 0]))

[1. 1. 0.]


In [18]:
#dislocation_name = 'A1--Cu--fcc--a-2-110--0-screw--{111}'
#dislocation_name = 'A1--Cu--fcc--a-2-110--90-edge--{100}'
#dislocation_name = 'A1--Cu--fcc--a-2-110--90-edge--{111}'

# Define dislocation_fi;e using librarydir and dislocation_name
#dislocation_file = library.get_ref(style='dislocation', name=dislocation_name)

# Create dictionary of input parameters related to the defect
inputs = {'ucell':ucell, 'dislocation_file':dislocation_file}

In [19]:
inputs

{'ucell': <atomman.core.System.System at 0x1f728900748>,
 'dislocation_file': DataModelDict([('dislocation',
                 DataModelDict([('key', 'unset'),
                                ('id',
                                 'A3--Mg--hcp--a-3-1120--90-edge--{0001}'),
                                ('character', 'edge'),
                                ('Burgers-vector', 'a/3[ 1, 1, -2, 0]'),
                                ('slip-plane', [0, 0, 0, 1]),
                                ('line-direction', [1, -1, 0, 0]),
                                ('system-family', 'A3--Mg--hcp'),
                                ('calculation-parameter',
                                 DataModelDict([('stroh_m', '0 1 0'),
                                                ('stroh_n', '0 0 1'),
                                                ('lineboxvector', 'a'),
                                                ('a_uvw', ' 1 -1  0'),
                                                ('b_uvw', ' 1 

#### 2.6 Specify calculation-specific run parameters

Simple input parameters:

- __boundarywidth__ sets the minimum width of the fixed-atom boundary region. This is taken in units of the a lattice parameter of the unit cell.

- __annealtemperature__ is the temperature at which to relax (anneal) the dislocation system. If annealtemperature is 0.0, then only a static relaxation will be performed. Default value is 0.0.

- __annealsteps__ is the number of nvt iteration steps to perform at the given temperature.  Default value is 0 if annealtemperature is zero, 10000 otherwise.

- __randomseed__ allows for the random seed used in generating initial atomic velocities for a dynamic relaxation to be specified. This is an integer between 1 and 900000000. Default value is None, which will randomly pick a number in that range.

- __duplicatecutoff__ Cutoff distance to use for identifying duplicate atoms to remove.  For dislocations with an edge component, applying the displacements creates an extra half-plane of atoms that will have (nearly) identical positions with other atoms after altering the boundary conditions.  Default cutoff value is 0.5 Angstrom.

- __onlyuselinear__ boolean flag. If False (default) the atoms in the middle of the system will be displaced according to an elasticity solution for a dislocation core and boundary atoms displaced by a linear gradient.  If True, all atoms are displaced by the linear gradient. 

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units. 

In [20]:
boundarywidth = 3 * ucell.box.a

annealtemperature = 50.0
annealsteps = 10000
randomseed = None

duplicatecutoff = uc.set_in_units(0.5, 'angstrom')
onlyuselinear = False

energytolerance = 0.0
forcetolerance = uc.set_in_units(1e-6, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

#### 2.7 Process defect input parameters 

Here, the defect parameters are read in and processed.

In [21]:
iprPy.input.subset('dislocation').interpret(inputs)

# Show inputs
inputs

{'ucell': <atomman.core.System.System at 0x1f728900748>,
 'dislocation_file': DataModelDict([('dislocation',
                 DataModelDict([('key', 'unset'),
                                ('id',
                                 'A3--Mg--hcp--a-3-1120--90-edge--{0001}'),
                                ('character', 'edge'),
                                ('Burgers-vector', 'a/3[ 1, 1, -2, 0]'),
                                ('slip-plane', [0, 0, 0, 1]),
                                ('line-direction', [1, -1, 0, 0]),
                                ('system-family', 'A3--Mg--hcp'),
                                ('calculation-parameter',
                                 DataModelDict([('stroh_m', '0 1 0'),
                                                ('stroh_n', '0 0 1'),
                                                ('lineboxvector', 'a'),
                                                ('a_uvw', ' 1 -1  0'),
                                                ('b_uvw', ' 1 

#### 2.8. Modify system

- __sizemults__ list of three sets of two integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in positive and negative directions when creating system.

- __a_uvw, b_uvw, c_uvw__ specifies how to orient the system by defining the crystal vectors of ucell to place along the three system box vectors.
    
- __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.

- __system__ is an atomman.System to insert the dislocation into. 

- __axes__ specifies the transformation matrix used to rotate the ucell to the system's orientation.

In [22]:
sizemults = '0 1 -50 50 -10 10'

inputs['sizemults'] = sizemults
iprPy.input.subset('atomman_systemmanipulate').interpret(inputs)

system = inputs['initialsystem']
print('# of atoms in system =', system.natoms)

axes = inputs['transformationmatrix']

# of atoms in system = 8000


In [23]:
sizemults = '0 1 -50 50 -10 10'

inputs['sizemults'] = sizemults
iprPy.input.subset('atomman_systemmanipulate').interpret(inputs)

system = inputs['initialsystem']
print('# of atoms in system =', system.natoms)

axes = inputs['transformationmatrix']

# of atoms in system = 8000


In [24]:
print(system.box)

avect =  [ 5.517,  0.000,  0.000]
bvect =  [ 0.000, 318.530,  0.000]
cvect =  [ 0.000,  0.000, 103.706]
origin = [ 0.000, -159.265, -51.853]


### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. dislarray_relax.template

In [25]:
with open('dislarray_relax.template', 'w') as f:
    f.write("""#LAMMPS input script for relaxing a periodic array of dislocations system

<atomman_system_info>

<atomman_pair_info>

# Simulation input parameters
variable bwidth equal <bwidth>

# Derived variables
variable topcut equal zhi-${bwidth}
variable botcut equal zlo+${bwidth}

# Define boundary groups
region rtop block INF INF INF INF ${topcut} INF
region rbot block INF INF INF INF INF ${botcut}
group top region rtop
group bot region rbot
group mid subtract all top bot
group bounds union top bot

compute peatom all pe/atom

dump first all custom <maxeval> *.dump id type x y z c_peatom
dump_modify first format <dump_modify_format>
thermo_style custom step pe

fix boundary bounds setforce 0.0 0.0 0.0

<anneal_info>

min_modify dmax <dmax>

minimize <etol> <ftol> <maxiter> <maxeval>""")

#### 3.2. dislocationarray()

In [26]:
def dislocationarray(lammps_command, system, potential, burgers,
                     C, mpi_command=None, axes=None, m=[0,1,0], n=[0,0,1],
                     etol=0.0, ftol=0.0, maxiter=10000, maxeval=100000, dmax=None,
                     annealtemp=0.0, annealsteps=None, randomseed=None,
                     bwidth=None, cutoff=None, onlyuselinear=False):
    """
    Creates and relaxes a dislocation monopole system.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The bulk system to add the defect to.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    burgers : list or numpy.array of float
        The burgers vector for the dislocation being added.
    C : atomman.ElasticConstants
        The system's elastic constants.
    mpi_command : str or None, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    axes : numpy.array of float or None, optional
        The 3x3 axes used to rotate the system by during creation.  If given,
        will be used to transform burgers and C from the standard
        crystallographic orientations to the system's Cartesian units.
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  Default is None which will select a
        random int between 1 and 900000000.
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of minimization iterations to use (default is 
        10000).
    maxeval : int, optional
        The maximum number of minimization evaluations to use (default is 
        100000).
    dmax : float, optional
        The maximum distance in length units that any atom is allowed to relax
        in any direction during a single minimization iteration (default is
        0.01 Angstroms).
    annealtemp : float, optional
        The temperature to perform a dynamic relaxation at. Default is 0.0,
        which will skip the dynamic relaxation.
    annealsteps : int, optional
        The number of time steps to run the dynamic relaxation for.  Default
        is None, which will run for 10000 steps if annealtemp is not 0.0. 
    bshape : str, optional
        The shape to make the boundary region.  Options are 'circle' and
        'rect' (default is 'circle').
    cutoff : float, optional
        Cutoff distance to use for identifying duplicate atoms to remove.
        For dislocations with an edge component, applying the displacements
        creates an extra half-plane of atoms that will have (nearly) identical
        positions with other atoms after altering the boundary conditions.
        Default cutoff value is 0.5 Angstrom.
    useonlylinear : bool, optional
        If False (default) the atoms in the middle of the system will be
        displaced according to an elasticity solution for a dislocation core
        and boundary atoms displaced by a linear gradient.  If True, all 
        atoms are displaced by the linear gradient.
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'dumpfile_base'** (*str*) - The filename of the LAMMPS dump file
          for the relaxed base system.
        - **'symbols_base'** (*list of str*) - The list of element-model
          symbols for the Potential that correspond to the base system's
          atypes.
        - **'Stroh_preln'** (*float*) - The pre-logarithmic factor in the 
          dislocation's self-energy expression.
        - **'Stroh_K_tensor'** (*numpy.array of float*) - The energy
          coefficient tensor based on the dislocation's Stroh solution.
        - **'dumpfile_disl'** (*str*) - The filename of the LAMMPS dump file
          for the relaxed dislocation monopole system.
        - **'symbols_disl'** (*list of str*) - The list of element-model
          symbols for the Potential that correspond to the dislocation
          monopole system's atypes.
        - **'E_total_disl'** (*float*) - The total potential energy of the
          dislocation monopole system.
    """
    try:
        # Get script's location if __file__ exists
        script_dir = Path(__file__).parent
    except:
        # Use cwd otherwise
        script_dir = Path.cwd()

    # Set default values
    if dmax is None:
        dmax = uc.set_in_units(0.01, 'angstrom')

    # Save base system
    system.dump('atom_dump', f='base.dump')

    # Generate periodic array of dislocations
    if onlyuselinear is False:
        dislsol = am.defect.solve_volterra_dislocation(C, burgers, axes=axes, m=m, n=n)
        dislsystem = am.defect.dislocation_array(system, dislsol=dislsol, bwidth=bwidth,
                                                 cutoff=cutoff)
    else:
        if axes is not None:
            T = am.tools.axes_check(axes)
            burgers = T.dot(burgers)
        dislsystem = am.defect.dislocation_array(system, burgers=burgers, m=m, n=n, cutoff=cutoff)

    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Get lammps version date
    lammps_date = lmp.checkversion(lammps_command)['date']
    
    # Define lammps variables
    lammps_variables = {}
    system_info = dislsystem.dump('atom_data', f='initial.dat',
                                  units=potential.units,
                                  atom_style=potential.atom_style)
    lammps_variables['atomman_system_info'] = system_info
    lammps_variables['atomman_pair_info'] = potential.pair_info(dislsystem.symbols)
    lammps_variables['anneal_info'] = anneal_info(annealtemp, annealsteps, 
                                                  randomseed, potential.units)
    lammps_variables['etol'] = etol
    lammps_variables['ftol'] = uc.get_in_units(ftol, lammps_units['force'])
    lammps_variables['maxiter'] = maxiter
    lammps_variables['maxeval'] = maxeval
    lammps_variables['dmax'] = dmax
    lammps_variables['bwidth'] = uc.get_in_units(bwidth, lammps_units['length'])
    
    # Set dump_modify format based on dump_modify_version
    if lammps_date < datetime.date(2016, 8, 3):
        lammps_variables['dump_modify_format'] = '"%d %d %.13e %.13e %.13e %.13e"'
    else:
        lammps_variables['dump_modify_format'] = 'float %.13e'
    
    # Write lammps input script
    template_file = Path(script_dir, 'dislarray_relax.template')
    lammps_script = 'dislarray_relax.in'
    with open(template_file) as f:
        template = f.read()
    with open(lammps_script, 'w') as f:
        f.write(iprPy.tools.filltemplate(template, lammps_variables,
                                         '<', '>'))
    
    # Run LAMMPS
    output = lmp.run(lammps_command, lammps_script, mpi_command)
    thermo = output.simulations[-1]['thermo']
    
    # Extract output values
    results_dict = {}
    results_dict['logfile'] = 'log.lammps'
    results_dict['dumpfile_base'] = 'base.dump'
    results_dict['symbols_base'] = system.symbols
    results_dict['dumpfile_disl'] = '%i.dump' % thermo.Step.values[-1]
    results_dict['symbols_disl'] = dislsystem.symbols

    return results_dict



#### 3.3 anneal_info() 

In [27]:
def anneal_info(temperature=0.0, runsteps=None, randomseed=None, units='metal'):
    """
    Generates LAMMPS commands for thermo anneal.
    
    Parameters
    ----------
    temperature : float, optional
        The temperature to relax at (default is 0.0).
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    units : str, optional
        The LAMMPS units style to use (default is 'metal').
    
    Returns
    -------
    str
        The generated LAMMPS input lines for performing a dynamic relax.
        Will be '' if temperature==0.0.
    """    
    # Return nothing if temperature is 0.0 (don't do thermo anneal)
    if temperature == 0.0:
        return ''
    
    # Generate velocity, fix nvt, and run LAMMPS command lines
    else:
        if randomseed is None:
            randomseed = random.randint(1, 900000000)
        if runsteps is None:
            runsteps = 10000
        
        start_temp = 2 * temperature
        tdamp = 100 * lmp.style.timestep(units)
        timestep = lmp.style.timestep(units)
        info = '\n'.join([
            'velocity mid create %f %i mom yes rot yes dist gaussian' % (start_temp, randomseed),
            'fix nvt all nvt temp %f %f %f' % (temperature, temperature,
                                               tdamp),
            'timestep %f' % (timestep),
            'thermo %i' % (runsteps),
            'run %i' % (runsteps),
            ])
    
    return info



### 4. Run calculation function(s)

In [28]:
results_dict = dislocationarray(lammps_command, system, potential, inputs['burgersvector'], C,
                                axes = axes,
                                mpi_command = mpi_command,
                                m = inputs['stroh_m'],
                                n = inputs['stroh_n'],
                                etol = energytolerance,
                                ftol = forcetolerance,
                                maxiter = maxiterations,
                                maxeval = maxevaluations,
                                dmax = maxatommotion,
                                annealtemp = annealtemperature, 
                                annealsteps = annealsteps,
                                randomseed = randomseed,
                                bwidth = boundarywidth,
                                cutoff = duplicatecutoff,
                                onlyuselinear = onlyuselinear)

### 5. Report results

#### 5.1 List dump files

In [29]:
print('Perfect system is saved as', results_dict['dumpfile_base'])
print('Defect system is saved as',  results_dict['dumpfile_disl'])

Perfect system is saved as base.dump
Defect system is saved as 10808.dump
